In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy, ks_2samp, jensenshannon
from scipy.special import rel_entr
import json
import zipfile
import os

def divergence_calculations(data, each_row=True):
    # Function to calculate KL divergence
    def kl_divergence(p, q):
        return np.sum(rel_entr(p, q))

    # Function to calculate Geometric Jensen-Shannon Divergence
    def geometric_jsd(p, q):
        jsd = jensenshannon(p, q)
        return np.sqrt(jsd)

    # Function to calculate Kolmogorov-Smirnov Test
    def ks_test(p, q):
        return ks_2samp(p, q).statistic

    results = []

    for index, row in data.iterrows():
        p = [row['methylated'], row['unmethylated']]
        q = [row['unmethylated'], row['methylated']]
        
        kl = kl_divergence(p, q)
        js = jensenshannon(p, q)
        gjs = geometric_jsd(p, q)
        ks = ks_test(p, q)

        row_result = {
            'index': index,
            'entropy': entropy(p),
            'relative_entropy': rel_entr(p, q).sum(),
            'dkl_divergence': kl,
            'jsd': js,
            'geometric_jsd': gjs,
            'kolmogorov_smirnov': ks
        }

        results.append(row_result)

    if each_row:
        # Save the results to a JSON file
        with open('results.json', 'w') as f:
            json.dump(results, f)
        
        # Zip the JSON file
        with zipfile.ZipFile('results.zip', 'w') as zipf:
            zipf.write('results.json')
        
        # Clean up the JSON file
        os.remove('results.json')
    else:
        # Calculate the statistics for the entire dataset
        p = data[['methylated', 'unmethylated']].sum().values
        q = data[['unmethylated', 'methylated']].sum().values

        kl = kl_divergence(p, q)
        js = jensenshannon(p, q)
        gjs = geometric_jsd(p, q)
        ks = ks_test(p, q)

        dataset_result = {
            'entropy': entropy(p),
            'relative_entropy': rel_entr(p, q).sum(),
            'dkl_divergence': kl,
            'jsd': js,
            'geometric_jsd': gjs,
            'kolmogorov_smirnov': ks
        }
        return dataset_result

# Example usage:
# Assuming the input data is in a CSV file
data = pd.read_csv('data.csv', sep='\t', header=None, names=['chr', 'start', 'end', 'percentage', 'methylated', 'unmethylated'])
results = divergence_calculations(data, each_row=True)


In [ ]:
from scipy.spatial.distance import jensenshannon


In [31]:
import pandas as pd
import numpy as np
from scipy.stats import entropy, ks_2samp
from scipy.special import rel_entr
from statsmodels.nonparametric.smoothers_lowess import lowess
import json
import zipfile
import os
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

def kl_divergence(p, q):
    return np.sum(rel_entr(p, q))

def geometric_jsd(p, q):
    jsd = jensenshannon(p, q)
    return np.sqrt(jsd)

def ks_test(p, q):
    return ks_2samp(p, q).statistic

def moving_average(data, window_size=3):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

def gaussian_smoothing(data, sigma=1):
    return np.exp(-np.square(data - np.mean(data)) / (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))

def laplace_smoothing(data, alpha=1):
    return (data + alpha) / (np.sum(data) + alpha * len(data))

def good_turing_smoothing(data):
    unique, counts = np.unique(data, return_counts=True)
    freq_of_freqs = np.bincount(counts)
    smoothed = np.array([((c+1) * (freq_of_freqs[c+1]/freq_of_freqs[c]) if c+1 < len(freq_of_freqs) else c) for c in counts])
    return smoothed / np.sum(smoothed)

def lowess_smoothing(data, frac=0.3):
    return lowess(data, np.arange(len(data)), frac=frac)[:, 1]

def hmm_smoothing(data, n_components=2):
    model = GaussianMixture(n_components=n_components)
    data_reshaped = data.reshape(-1, 1)
    model.fit(data_reshaped)
    smoothed = model.predict_proba(data_reshaped)[:, 1]
    return smoothed

def apply_smoothing(data, method):
    if method == 'moving_average':
        return moving_average(data)
    elif method == 'gaussian':
        return gaussian_smoothing(data)
    elif method == 'laplace':
        return laplace_smoothing(data)
    elif method == 'good_turing':
        return good_turing_smoothing(data)
    elif method == 'lowess':
        return lowess_smoothing(data)
    elif method == 'hmm':
        return hmm_smoothing(data)
    else:
        raise ValueError(f"Unknown smoothing method: {method}")

def divergence_calculations(data, each_row=True, smoothing='laplace'):
    results = []

    if each_row:
        for index, row in data.iterrows():
            p = np.array([row['methylated'], row['unmethylated']], dtype=np.float64)
            q = np.array([row['unmethylated'], row['methylated']], dtype=np.float64)
            
            # Apply smoothing
            p = apply_smoothing(p, smoothing)
            q = apply_smoothing(q, smoothing)
            
            # Normalize p and q
            p /= p.sum()
            q /= q.sum()
            
            kl = kl_divergence(p, q)
            js = jensenshannon(p, q)
            gjs = geometric_jsd(p, q)
            ks = ks_test(p, q)

            row_result = {
                'index': index,
                'entropy': entropy(p),
                'relative_entropy': kl,  # KL divergence is relative entropy
                'jsd': js,
                'geometric_jsd': gjs,
                'kolmogorov_smirnov': ks
            }

            results.append(row_result)

        # Save the results to a JSON file
        with open('results.json', 'w') as f:
            json.dump(results, f)
        
        # Zip the JSON file
        with zipfile.ZipFile('results.zip', 'w') as zipf:
            zipf.write('results.json')
        
        # Clean up the JSON file
        os.remove('results.json')
    else:
        # Calculate the statistics for the entire dataset
        p = data[['methylated', 'unmethylated']].values.flatten().astype(np.float64)
        q = data[['unmethylated', 'methylated']].values.flatten().astype(np.float64)

        # Apply smoothing
        p = apply_smoothing(p, smoothing)
        q = apply_smoothing(q, smoothing)

        # Normalize p and q
        p /= p.sum()
        q /= q.sum()

        kl = kl_divergence(p, q)
        js = jensenshannon(p, q)
        gjs = geometric_jsd(p, q)
        ks = ks_test(p, q)

        dataset_result = {
            'entropy': entropy(p),
            'relative_entropy': kl,  # KL divergence is relative entropy
            'jsd': js,
            'geometric_jsd': gjs,
            'kolmogorov_smirnov': ks
        }
        return dataset_result

# Sample data
data = pd.DataFrame({
    'chr': [1]*11,
    'start': [5694710, 5694712, 5694725, 5694727, 5694740, 5694742, 5694743, 5694745, 5694751, 5694753, 5694776],
    'end': [5694710, 5694712, 5694725, 5694727, 5694740, 5694742, 5694743, 5694745, 5694751, 5694753, 5694776],
    'percentage': [33.33, 0, 0, 66.67, 33.33, 33.33, 0, 66.67, 33.33, 0, 44.44],
    'methylated': [1, 0, 0, 2, 1, 1, 0, 2, 1, 0, 4],
    'unmethylated': [2, 2, 2, 1, 2, 2, 3, 1, 2, 4, 5]
})

# Test for each row with different smoothing methods
smoothing_methods = ['moving_average', 'gaussian', 'laplace', 'lowess', 'hmm']
results = {}
zip_file = 'results.zip'

for method in smoothing_methods:
    print(f"Testing method: {method}")
    result = divergence_calculations(data, each_row=False, smoothing=method)
    print(result)
    print("''##################################S")
    




Testing method: moving_average
{'entropy': 2.9049214167300583, 'relative_entropy': 0.08399291238911544, 'jsd': 0.14535312126105104, 'geometric_jsd': 0.38125204427130754, 'kolmogorov_smirnov': 0.15}
''##################################S
Testing method: gaussian
{'entropy': 2.887980776677277, 'relative_entropy': 0.21172773363454295, 'jsd': 0.22091664414062753, 'geometric_jsd': 0.47001770619906175, 'kolmogorov_smirnov': 0.0}
''##################################S
Testing method: laplace
{'entropy': 2.976434987573053, 'relative_entropy': 0.29343660155352397, 'jsd': 0.26189433849794413, 'geometric_jsd': 0.5117561318615969, 'kolmogorov_smirnov': 0.36363636363636365}
''##################################S
Testing method: good_turing
{'entropy': 0.639031859650177, 'relative_entropy': 0.0, 'jsd': 0.0, 'geometric_jsd': 0.0, 'kolmogorov_smirnov': 0.0}
''##################################S
Testing method: lowess
{'entropy': 2.976684645125091, 'relative_entropy': 0.02318320340661692, 'jsd': 0.0752901

In [28]:
import json
import zipfile
import os

def read_json_from_zip(zip_file, json_file):
    # Unzip the JSON file
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall()
    
    # Read the JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Clean up the extracted JSON file
    os.remove(json_file)
    
    return data

# Usage
zip_file = 'results.zip'
json_file = 'results.json'
data = read_json_from_zip(zip_file, json_file)

# Print the data
print(data)


[{'index': 0, 'entropy': 0.0, 'relative_entropy': 0.0, 'jsd': 0.0, 'geometric_jsd': 0.0, 'kolmogorov_smirnov': 0.0}, {'index': 1, 'entropy': 0.0, 'relative_entropy': inf, 'jsd': 0.8325546111576977, 'geometric_jsd': 0.9124443057840285, 'kolmogorov_smirnov': 0.0}, {'index': 2, 'entropy': 0.0, 'relative_entropy': 0.0, 'jsd': 0.0, 'geometric_jsd': 0.0, 'kolmogorov_smirnov': 0.0}, {'index': 3, 'entropy': 0.0, 'relative_entropy': inf, 'jsd': 0.8325546111576977, 'geometric_jsd': 0.9124443057840285, 'kolmogorov_smirnov': 0.0}, {'index': 4, 'entropy': 0.0, 'relative_entropy': 0.0, 'jsd': 0.0, 'geometric_jsd': 0.0, 'kolmogorov_smirnov': 0.0}, {'index': 5, 'entropy': 0.0, 'relative_entropy': 0.0, 'jsd': 0.0, 'geometric_jsd': 0.0, 'kolmogorov_smirnov': 0.0}, {'index': 6, 'entropy': 0.0, 'relative_entropy': inf, 'jsd': 0.8325546111576977, 'geometric_jsd': 0.9124443057840285, 'kolmogorov_smirnov': 0.0}, {'index': 7, 'entropy': 0.0, 'relative_entropy': 0.0, 'jsd': 0.0, 'geometric_jsd': 0.0, 'kolmogor

In [18]:
for item in data.iterrows():
    print(item)

AttributeError: 'list' object has no attribute 'iterrows'